In [18]:
import torch
import cv2
import tqdm
import matplotlib as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [40]:
meta_file = '../data/HAM10k/HAM10000_metadata.csv'
meta = pd.read_csv(meta_file)
meta.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear


In [41]:
X, y = meta.iloc[:,[0,1,3,4,5,6]], meta.iloc[:,2]

In [42]:
xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.20, stratify=y)
print(f"train: {len(xtrain)}, test: {len(xtest)} ")

train: 8012, test: 2003 


In [43]:
ytrain.value_counts()

nv       5364
mel       890
bkl       879
bcc       411
akiec     262
vasc      114
df         92
Name: dx, dtype: int64

In [44]:
ytest.value_counts()

nv       1341
mel       223
bkl       220
bcc       103
akiec      65
vasc       28
df         23
Name: dx, dtype: int64

In [48]:
xtrain.insert(loc = 2, column = 'dx', value = ytrain)


,lesion_id,image_id,dx,dx_type,age,sex,localization
7744,HAM_0001943,ISIC_0034301,nv,histo,30.0,female,lower extremity
7834,HAM_0002898,ISIC_0033179,nv,histo,80.0,female,upper extremity
483,HAM_0005146,ISIC_0026526,bkl,histo,60.0,male,face
7836,HAM_0002898,ISIC_0033721,nv,histo,80.0,female,upper extremity
3478,HAM_0006569,ISIC_0030483,nv,follow_up,35.0,male,lower extremity
...,...,...,...,...,...,...,...
1860,HAM_0001506,ISIC_0033051,mel,histo,25.0,female,lower extremity
1617,HAM_0007180,ISIC_0033272,mel,histo,65.0,male,face
8513,HAM_0000129,ISIC_0027486,nv,histo,40.0,female,back
927,HAM_0006003,ISIC_0032827,bkl,consensus,85.0,female,lower extremity


In [49]:
xtest.insert(loc= 2 , column = 'dx', value = ytest )

In [52]:
root_path = '../data/HAM10k/HAM10000_images/'
xtrain['image_pth'] = root_path + xtrain['image_id'] + str('.jpg')


,lesion_id,image_id,dx,dx_type,age,sex,localization,image_pth
7744,HAM_0001943,ISIC_0034301,nv,histo,30.0,female,lower extremity,../data/HAM10k/HAM10000_images/ISIC_0034301.jpg
7834,HAM_0002898,ISIC_0033179,nv,histo,80.0,female,upper extremity,../data/HAM10k/HAM10000_images/ISIC_0033179.jpg
483,HAM_0005146,ISIC_0026526,bkl,histo,60.0,male,face,../data/HAM10k/HAM10000_images/ISIC_0026526.jpg
7836,HAM_0002898,ISIC_0033721,nv,histo,80.0,female,upper extremity,../data/HAM10k/HAM10000_images/ISIC_0033721.jpg
3478,HAM_0006569,ISIC_0030483,nv,follow_up,35.0,male,lower extremity,../data/HAM10k/HAM10000_images/ISIC_0030483.jpg
...,...,...,...,...,...,...,...,...
1860,HAM_0001506,ISIC_0033051,mel,histo,25.0,female,lower extremity,../data/HAM10k/HAM10000_images/ISIC_0033051.jpg
1617,HAM_0007180,ISIC_0033272,mel,histo,65.0,male,face,../data/HAM10k/HAM10000_images/ISIC_0033272.jpg
8513,HAM_0000129,ISIC_0027486,nv,histo,40.0,female,back,../data/HAM10k/HAM10000_images/ISIC_0027486.jpg
927,HAM_0006003,ISIC_0032827,bkl,consensus,85.0,female,lower extremity,../data/HAM10k/HAM10000_images/ISIC_0032827.jpg


In [53]:
xtest['image_pth'] = root_path + xtest['image_id'] + str('.jpg')

In [55]:
xtest.to_csv('../data/test.csv', index=False)
xtrain.to_csv('../data/train.csv', index=False)